## Laboratorio 2

### Andre Marroquin Tarot- 22266

### Sergio Orellana- 221122

### Rodrigo Mansilla 22611

LINK GIT: https://github.com/mar22266/LABORATORIOS-IA.git

# Task 1 - Preguntas Teóricas

1. ¿Por qué el modelo de Naive Bayes se le considera “naive”?

    R//


2. Explique la formulación matemática que se busca optimizar en Support Vector Machine, además responda
¿cómo funciona el truco del Kernel para este modelo? (Lo que se espera de esta pregunta es que puedan
explicar en sus propias palabras la fórmula a la que llegamos que debemos optimizar de SVM en clase)

    R//


3. Investigue sobre Random Forest y responda

    a. ¿Qué tipo de ensemble learning es este modelo?

        R//

    b. ¿Cuál es la idea general detrás de Random Forest?

        R//

    c. ¿Por qué se busca baja correlación entre los árboles de Random Forest?

        R//

# Task 2 y 2.2

In [11]:
# Importar Librerías Necesarias

import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Leer y Limpiar el Dataset

def cargar_y_limpiar_dataset(ruta):
    mensajes = []
    etiquetas = []

    with open(ruta, 'r', encoding='utf-8') as archivo:
        for linea in archivo:
            linea = linea.strip()  # Quitar espacios en blanco iniciales y finales
            if not linea:
                continue
            etiqueta, mensaje = linea.split('\t', 1)  # Dividir etiqueta y mensaje
            etiqueta = etiqueta.lower()  # Normalizar etiquetas a minúsculas
            mensaje = re.sub(r'[^a-zA-Z0-9\s]', '', mensaje).lower()  # Quitar caracteres especiales
            mensajes.append(mensaje)
            etiquetas.append(1 if etiqueta == 'spam' else 0)  # 1 para spam, 0 para ham

    return mensajes, etiquetas

# Cargar y limpiar el dataset
ruta_dataset = 'entrenamiento.txt'
mensajes, etiquetas = cargar_y_limpiar_dataset(ruta_dataset)

In [3]:
#  Dividir en Conjuntos de Entrenamiento, Prueba y Validación
X_train, X_temp, y_train, y_temp = train_test_split(mensajes, etiquetas, test_size=0.2, random_state=42)  # 80% entrenamiento
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 10% validación, 10% prueba

In [4]:
# Implementar Naive Bayes con Laplace Smoothing

def calcular_probabilidades(X, y):
    # Calcular probabilidades de Naive Bayes con Laplace Smoothing
    vocabulario = set(word for mensaje in X for word in mensaje.split())
    vocab_size = len(vocabulario)

    # Inicializar conteos y probabilidades
    conteo_palabras = {clase: {} for clase in [0, 1]}
    total_palabras = {clase: 0 for clase in [0, 1]}
    prob_clase = {}

    for clase in [0, 1]:
        mensajes_clase = [X[i] for i in range(len(X)) if y[i] == clase]
        total_mensajes = len(mensajes_clase)
        prob_clase[clase] = total_mensajes / len(y)

        for mensaje in mensajes_clase:
            for palabra in mensaje.split():
                if palabra not in conteo_palabras[clase]:
                    conteo_palabras[clase][palabra] = 0
                conteo_palabras[clase][palabra] += 1
                total_palabras[clase] += 1

    return vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase

# Calcular probabilidades para el modelo
vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase = calcular_probabilidades(X_train, y_train)

def predecir(X, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase):
    # Predecir etiquetas para nuevos mensajes
    predicciones = []

    for mensaje in X:
        probabilidades = {}
        for clase in [0, 1]:
            probabilidad = np.log(prob_clase[clase])
            for palabra in mensaje.split():
                conteo = conteo_palabras[clase].get(palabra, 0)
                probabilidad += np.log((conteo + 1) / (total_palabras[clase] + vocab_size))
            probabilidades[clase] = probabilidad

        predicciones.append(max(probabilidades, key=probabilidades.get))

    return predicciones


In [5]:
# Evaluar el Modelo
y_pred = predecir(X_test, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase)

# Métricas de desempeño
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
reporte = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", reporte)


Accuracy: 0.9784560143626571
Confusion Matrix:
 [[472  10]
 [  2  73]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       482
           1       0.88      0.97      0.92        75

    accuracy                           0.98       557
   macro avg       0.94      0.98      0.96       557
weighted avg       0.98      0.98      0.98       557



# Task 2.2

In [6]:
# Construcción del Modelo

def calcular_probabilidades(X, y):
    # Calcular probabilidades de Naive Bayes con Laplace Smoothing
    vocabulario = set(word for mensaje in X for word in mensaje.split())
    vocab_size = len(vocabulario)

    # Inicializar conteos y probabilidades
    conteo_palabras = {clase: {} for clase in [0, 1]}
    total_palabras = {clase: 0 for clase in [0, 1]}
    prob_clase = {}

    for clase in [0, 1]:
        mensajes_clase = [X[i] for i in range(len(X)) if y[i] == clase]
        total_mensajes = len(mensajes_clase)
        prob_clase[clase] = total_mensajes / len(y)

        for mensaje in mensajes_clase:
            for palabra in mensaje.split():
                if palabra not in conteo_palabras[clase]:
                    conteo_palabras[clase][palabra] = 0
                conteo_palabras[clase][palabra] += 1
                total_palabras[clase] += 1

    return vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase

In [7]:
# Calcular probabilidades usando solo el conjunto de entrenamiento
vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase = calcular_probabilidades(X_train, y_train)

def predecir(X, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase):
    """Predecir etiquetas para nuevos mensajes."""
    predicciones = []

    for mensaje in X:
        probabilidades = {}
        for clase in [0, 1]:
            probabilidad = np.log(prob_clase[clase])
            for palabra in mensaje.split():
                conteo = conteo_palabras[clase].get(palabra, 0)
                probabilidad += np.log((conteo + 1) / (total_palabras[clase] + vocab_size))
            probabilidades[clase] = probabilidad

        predicciones.append(max(probabilidades, key=probabilidades.get))

    return predicciones


In [8]:
# Evaluación del Modelo
# Predecir sobre el conjunto de entrenamiento
y_train_pred = predecir(X_train, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase)

# Predecir sobre el conjunto de prueba
y_test_pred = predecir(X_test, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase)

# Calcular métricas de desempeño
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
conf_matrix_train = confusion_matrix(y_train, y_train_pred)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)

print("Accuracy en el conjunto de entrenamiento:", accuracy_train)
print("Matriz de confusión en el conjunto de entrenamiento:\n", conf_matrix_train)
print("Accuracy en el conjunto de prueba:", accuracy_test)
print("Matriz de confusión en el conjunto de prueba:\n", conf_matrix_test)


Accuracy en el conjunto de entrenamiento: 0.9912398921832885
Matriz de confusión en el conjunto de entrenamiento:
 [[3854   14]
 [  25  559]]
Accuracy en el conjunto de prueba: 0.9784560143626571
Matriz de confusión en el conjunto de prueba:
 [[472  10]
 [  2  73]]


## Justificación de la métrica utilizada
## Accuracy es una métrica adecuada dado que el dataset tiene una distribución balanceada entre clases ham y spam

# Task 2.3

In [10]:
# Clasificación de Mensajes Futuros
def clasificar_mensaje(mensaje, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase):
    """Clasificar un mensaje individual como spam o ham y devolver las probabilidades."""
    mensaje_original = mensaje  # Guardar mensaje original
    mensaje = re.sub(r'[^a-zA-Z0-9\s]', '', mensaje).lower()  # Limpiar mensaje
    probabilidades = {}
    for clase in [0, 1]:
        probabilidad = np.log(prob_clase[clase])
        for palabra in mensaje.split():
            conteo = conteo_palabras[clase].get(palabra, 0)
            probabilidad += np.log((conteo + 1) / (total_palabras[clase] + vocab_size))
        probabilidades[clase] = probabilidad

    clase_predicha = max(probabilidades, key=probabilidades.get)
    return mensaje_original, probabilidades, "spam" if clase_predicha == 1 else "ham"

# Interfaz para clasificar mensajes nuevos
while True:
    mensaje = input("Ingrese un mensaje para clasificar (o 'salir' para terminar): ")
    if mensaje.lower() == 'salir':
        break
    mensaje_original, probabilidades, clasificacion = clasificar_mensaje(mensaje, vocabulario, vocab_size, conteo_palabras, total_palabras, prob_clase)
    print(f"Mensaje ingresado: {mensaje_original}")
    # La probabilidad logarítmica de que el mensaje pertenezca a la clase ham(0) o spam(1)
    print(f"Probabilidades: {probabilidades}")
    print(f"Clasificación: {clasificacion}")

Mensaje ingresado: hola me llamo andre
Probabilidades: {0: -37.190714021335005, 1: -38.7920862010779}
Clasificación: ham
Mensaje ingresado: ATENCION
Probabilidades: {0: -11.178147188857753, 1: -12.045386563945964}
Clasificación: ham
Mensaje ingresado: ingresa tu numero ya
Probabilidades: {0: -40.41954017705637, 1: -41.39477588652228}
Clasificación: ham
Mensaje ingresado: promocion de dos por 1
Probabilidades: {0: -49.009304407253325, 1: -48.76989739128578}
Clasificación: spam
Mensaje ingresado: 
Probabilidades: {0: -0.14061585582225045, 1: -2.031207729567208}
Clasificación: ham


# Task 2.4

In [12]:
# Comparación con Librerías
# Entrenamiento con sklearn.naive_bayes.MultinomialNB
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

modelo_sklearn = MultinomialNB()
modelo_sklearn.fit(X_train_vect, y_train)

# Predicción sobre el conjunto de entrenamiento y prueba
y_train_pred_sklearn = modelo_sklearn.predict(X_train_vect)
y_test_pred_sklearn = modelo_sklearn.predict(X_test_vect)

# Métricas de desempeño para sklearn
accuracy_train_sklearn = accuracy_score(y_train, y_train_pred_sklearn)
accuracy_test_sklearn = accuracy_score(y_test, y_test_pred_sklearn)

print("\nResultados utilizando sklearn.naive_bayes.MultinomialNB")
print("Accuracy en el conjunto de entrenamiento:", accuracy_train_sklearn)
print("Accuracy en el conjunto de prueba:", accuracy_test_sklearn)


Resultados utilizando sklearn.naive_bayes.MultinomialNB
Accuracy en el conjunto de entrenamiento: 0.9914645103324349
Accuracy en el conjunto de prueba: 0.9838420107719928




## ¿Cuál implementación lo hizo mejor?

La implementación de la librería **sklearn** fue superior en términos de desempeño comparado con la implementación manual:

- **Entrenamiento:**
  - **sklearn:** Obtuvó una precisión de 0.9915
  - **Manual:** Obtuvo una precisión de 0.9912

- **Prueba:**
  - **sklearn:** Alcanzó 0.9838
  - **Manual:** Obtuvo 0.9785



## ¿Por qué cree que se debe esta diferencia?

La diferencia en el desempeño entre sklearn y la implementación manual se puede deber a algunos factores:

   - La implementación de sklearn utiliza algoritmos optimizados que incluyen el manejo eficiente de datos y operaciones matemáticas en bajo nivel.

   - MultinomialNB maneja de forma nativa problemas como el underflow numérico y aplica ajustes internos que mejoran la estabilidad de los cálculos.

   - La herramienta CountVectorizer y el modelo de sklearn aseguran un tratamiento consistente del texto, lo que reduce posibles errores en la preparación manual.
